# Mobile App Statistics - Kaggle

## Background

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Instead of collecting data for over 4 million apps which requires a significant amount of time and money, there are two smaller sample datasets available:

- A data set containing data about approximately 10,000 Android apps from Google Play.
- A data set containing data about approximately 7,000 iOS apps from the App Store; the data was collected in July 2017.

[Kaggle - Documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)

## Problem Statement

The main source of revenue for free apps consists of in-app ads. This means the revenue for any given app is mostly influenced by the number of users who use our app — the more users that see and engage with the ads, the better. 

*The goal for this project is to analyze data to help the developers understand what type of apps are likely to attract more users.*

## Data Exploration

In [1]:
from csv import reader

open_file_Apple = open('AppleStore.csv', encoding='utf8')
read_file_Apple = reader(open_file_Apple)
Apple_data = list(read_file_Apple)
Apple_header = Apple_data[0]
Apple = Apple_data[1:]

open_file_Google = open('googleplaystore.csv', encoding='utf8')
read_file_Google = reader(open_file_Google)
Google_data = list(read_file_Google)
Google_header = Google_data[0]
Google = Google_data[1:]

To make it easier to explore the two data sets, we'll first write a function named *explore_data()* that we can use repeatedly to explore rows in a more readable way. We'll also add an option for our function to show the number of rows and columns for any data set.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

- Print the first few rows of each data set.
- Find the number of rows and columns of each data set.

In [3]:
print(Apple_header, '\n\n')
explore_data(Apple,0,3,True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


In [4]:
print(Google_header, '\n\n')
explore_data(Google,0,3,True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


- There are 7197 IOS apps in the dataset, and the columns that seem interesting are: 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'. 

- There are 10841 apps and 13 columns in the Google Play dataset. The columns that might be useful for the purpose of analysis are 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres'.

## Data Cleaning

As we are only interested in Apps that are free and are directed toward an English-speaking audience. This means that we'll need to:

- Remove apps that aren't free.
- Remove non-English apps.

### Remove Incorrect Data

The Google Play data set has a dedicated discussion section, and we can see that one of the discussions outlines an error for row 10472. Let's print this row and compare it against the header and another row that is correct.

In [5]:
print(Google_header,'\n')
print('Incorrect Row:\n', Google[10472],'\n')
print('Correct Row:\n', Google[0])  

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

Incorrect Row:
 ['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'] 

Correct Row:
 ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


The 10472th row is missing information for 'Category' thus I will either:
- delete this row.
- assign a category label for it based on other data.

For now, I choose to delete this row as it is the only row in the dataset (a small price to pay).

In [6]:
print(len(Google))
del Google[10472]
print(len(Google))

10841
10840


### Remove duplicates

Create a function that can be reused to check duplicates based on app names for both Apple & Google datasets.

In [48]:
def check_duplicates(dataset):
    unique_app = []
    duplicate_app = []
    
    for row in dataset:
        if dataset == Google or dataset == Google_clean:
            app = row[0]
        elif dataset == Apple or dataset == Apple_clean:
            app = row[1]
        else:
            return print("Undefined dataset: only Apple or Google accepted")
            
        if app in unique_app:
            duplicate_app.append(app)
        else:
            unique_app.append(app)
    
    print('Number of unique apps:', len(unique_app))
    print('Number of duplicate apps:', len(duplicate_app), '\n')
    
    if len(duplicate_app) >= 10:
        print(duplicate_app[:10])
    else:
        print(duplicate_app[:len(duplicate_app)])

In [49]:
check_duplicates(Apple)

Number of unique apps: 7195
Number of duplicate apps: 2 

['Mannequin Challenge', 'VR Roller Coaster']


In [50]:
check_duplicates(Google)

Number of unique apps: 9659
Number of duplicate apps: 1181 

['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


Apple dataset seems to be okay without many duplicates so I will leave them be for now, however, there are 1181 rows of duplicates in the Google dataset.

Let's check duplicates for *Instagram* app in the Google dataset to see what are the duplicates.

In [21]:
print(Google_header)
for item in Google:
    app = item[0]
    if app == 'Instagram':
        print(item)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


I need to remove the duplicate entries and keep only one entry per app to avoid skewing data representations. One thing I could do is remove the duplicate rows randomly, but probably there is a better way.

A closer examine on the rows printed above for the Instagram app, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show that the data was collected at different times. 

Instead of removing rows randomly, I will keep the rows that have the highest number of reviews because the higher the number of reviews, the more reliable or newer the ratings.

To do that, I will:

- Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app.
- Use the dictionary to create a new data set, which will have only one entry per app (and I only select the apps or row with the highest number of reviews)

In [31]:
reviews_max = {}
for item in Google:
    name = item[0]
    n_reviews = float(item[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
    else:
        pass
print('Number of unique data:', len(reviews_max))
print(list(reviews_max.items())[:10])

Number of unique data: 9659
[('Photo Editor & Candy Camera & Grid & ScrapBook', 159.0), ('Coloring book moana', 974.0), ('U Launcher Lite – FREE Live Cool Themes, Hide Apps', 87510.0), ('Sketch - Draw & Paint', 215644.0), ('Pixel Draw - Number Art Coloring Book', 967.0), ('Paper flowers instructions', 167.0), ('Smoke Effect Photo Maker - Smoke Editor', 178.0), ('Infinite Painter', 36815.0), ('Garden Coloring Book', 13791.0), ('Kids Paint Free - Drawing Fun', 121.0)]


Now, let's use the reviews_max dictionary to remove the duplicates. For the duplicate cases, I'll only keep the entries with the highest number of reviews. In the code cell below:

- Start by initializing two empty lists, Google_clean and added_app.
- Loop through the Google dataset, and for every iteration:
    - Isolate the name of the app and the number of reviews.
    - Add the current row to the Google_clean list, and the app name to the added_app list if:
        - The number of reviews of the current row matches the number of reviews of that app as described in the reviews_max dictionary; and
        - The name of the app is not already in the added_app list.

In [51]:
Google_clean = []
added_app = []
for item in Google:
    name = item[0]
    n_reviews = float(item[3])
    if name not in added_app and n_reviews == reviews_max[name]:
        Google_clean.append(item)
        added_app.append(name)

Now let's quickly explore the new data set, and confirm that the number of rows is 9,659.

In [52]:
explore_data(Google_clean,0,3,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


In [53]:
check_duplicates(Google_clean)

Number of unique apps: 9659
Number of duplicate apps: 0 

[]


### Remove non-English Apps

I'm not interested in apps not intended for English audiences, so I'll remove them. One way to go about this is to remove each app whose name contains a symbol that is not commonly used in English text which are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, so I can take advantage of that to build a function that checks an app name to see if it contains non-ASCII characters.

In [58]:
def english_check(a_string):
    for letter in a_string:
        if ord(letter) > 127: # >127 is not an ASCII character
            return False
    return True
print(english_check('Instagram'))
print(english_check('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english_check('Docs To Go™ Free Office Suite'))
print(english_check('Instachat 😜'))

True
False
False
False


The function seems to work fine, but some English app names use emojis or other symbols (™, — (em dash), – (en dash), etc.) that fall outside of the ASCII range. Because of this, some useful data can be lost.

To minimize the impact of data loss, I'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English. It's still not perfect, but it should be fairly effective.

In [59]:
def english_check(a_string):
    non_english_n = 0
    for letter in a_string:
        if ord(letter) > 127:
            non_english_n += 1
            if non_english_n > 3:
                return False
    return True
print(english_check('Instagram'))
print(english_check('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english_check('Docs To Go™ Free Office Suite'))
print(english_check('Instachat 😜'))

True
False
True
True


The function is still not perfect, and very few non-English apps might get past the filter, but this seems good enough at this point in the analysis — I will not spend too much time on optimization at this point. I will now apply the filter to clean the datasets.

In [60]:
Apple_new = []
Google_new = []

for item in Apple:
    if english_check(item[1]):
        Apple_new.append(item)

for item in Google_clean:
    if english_check(item[0]):
        Google_new.append(item)
        
explore_data(Apple_new,0,3,True)
print('\n')
explore_data(Google_new,0,3,True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 6183
Number of columns: 16


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Vari

There are 9614 Google apps and 6183 IOS apps left.

### Isolicating Free Apps

So far in the data cleaning process, I have:

- Removed incorrect data
- Removed duplicate app entries
- Removed non-English apps

Next time is to isolate the free apps.

In [64]:
Apple_final = []
Google_final = []

for item in Apple_new:
    price = item[4]
    if price == '0.0':
        Apple_final.append(item)
        
for item in Google_new:
    price = item[7]
    if price == '0':
        Google_final.append(item)
        
explore_data(Apple_final,0,3,True)
print('\n')
explore_data(Google_final,0,3,True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 3222
Number of columns: 16


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Vari

There are 8864 free Google apps and 3222 free IOS apps.

## Most Common Apps by Genre

To minimize risks and overhead, the validation strategy for an app idea is comprised of three steps:

- Build a minimal Google version of the app, and add it to Google Play.
- If the app has a good response from users, develop it further.
- If the app is profitable after six months, build an IOS version of the app and add it to the App Store.

Because the end goal is to add the app on both the App Store and Google Play, I need to find app profiles that are successful on both markets.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, I'll build a frequency table for the *prime_genre* column of the App Store dataset, and the *Genres* and *Category* columns of the Google Play dataset.

Build two functions to analyze the frequency tables:

- One function to generate frequency tables that show percentages.
- Another function to display the percentages in a descending order.

In [65]:
def freq_table(dataset, index):
    dic_dataset = {}
    for item in dataset:
        key = item[index]
        if key in dic_dataset:
            dic_dataset[key] += 1
        else:
            dic_dataset[key] = 1
    
    total_number = len(dataset)
    for key in dic_dataset:
        dic_dataset[key] /= total_number
        dic_dataset[key] *= 100
    return dic_dataset

In [70]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

Start by examining the frequency table for the prime_genre column (INDEX=11) of the App Store data set.

In [71]:
display_table(Apple_final, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare.

However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users: **the demand might not be the same as the offer**.

Moving on by examining the Genres (INDEX=-4) and Category (INDEX=1) columns of the Google Play data set (two columns which seem to be related).

In [73]:
# Category (INDEX=1)
display_table(Google_final, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, a closer look indicates that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table for the Genres column:

In [75]:
# Genres (INDEX=-4)
display_table(Google_final, -4)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

The difference between the Genres and the Category columns is not crystal clear, but one thing to notice is that the Genres column is much more granular (it has more categories). I want to look for the bigger picture at the moment, so I'll only work with the Category column moving forward.

So the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps.

Now I'd like to get an idea about the kind of apps that have most users.

## Most Popular Apps by Genre on App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play dataset, I can find this information in the *Installs* column, but for the App Store dataset this information is missing. As a workaround, I'll take the total number of user ratings as a proxy, which we can find in *rating_count_tot*.

To calculate the average number of user ratings per genre on the App Store:

In [82]:
# Frequency table for prime_genre but I only use the keys
genres_Apple = freq_table(Apple_final, -5)

for key in genres_Apple:
    total = 0
    len_genre = 0
    for item in Apple_final:
        if item[-5] == key:
            genre_app = float(item[5])
            total += genre_app
            len_genre += 1
    genres_Apple[key] = total / len_genre

In [84]:
genres_list = []
for key in genres_Apple:
    genres_tuple = (genres_Apple[key], key)
    genres_list.append(genres_tuple)

genres_list = sorted(genres_list, reverse=True)

for each in genres_list:
    print(each[1],':',each[0])

Navigation : 86090.33333333333
Reference : 74942.11111111111
Social Networking : 71548.34905660378
Music : 57326.530303030304
Weather : 52279.892857142855
Book : 39758.5
Food & Drink : 33333.92307692308
Finance : 31467.944444444445
Photo & Video : 28441.54375
Travel : 28243.8
Shopping : 26919.690476190477
Health & Fitness : 23298.015384615384
Sports : 23008.898550724636
Games : 22788.6696905016
News : 21248.023255813954
Productivity : 21028.410714285714
Utilities : 18684.456790123455
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Business : 7491.117647058823
Education : 7003.983050847458
Catalogs : 4004.0
Medical : 612.0


In [89]:
for each in Apple_final:
    if each[-5] == 'Navigation':
        print('App:',each[1],'\nTotal number ratings:',each[5],'\n')

App: Waze - GPS Navigation, Maps & Real-time Traffic 
Total number ratings: 345046 

App: Google Maps - Navigation & Transit 
Total number ratings: 154911 

App: Geocaching® 
Total number ratings: 12811 

App: CoPilot GPS – Car Navigation & Offline Maps 
Total number ratings: 3582 

App: ImmobilienScout24: Real Estate Search in Germany 
Total number ratings: 187 

App: Railway Route Search 
Total number ratings: 5 



On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together.

The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

The goal is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. To get a better picture, I could have removed these extremely popular apps for each genre and then rework the averages, but I'll leave this level of detail for later.

In [90]:
for each in Apple_final:
    if each[-5] == 'Reference':
        print('App:',each[1],'\nTotal number ratings:',each[5],'\n')

App: Bible 
Total number ratings: 985920 

App: Dictionary.com Dictionary & Thesaurus 
Total number ratings: 200047 

App: Dictionary.com Dictionary & Thesaurus for iPad 
Total number ratings: 54175 

App: Google Translate 
Total number ratings: 26786 

App: Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran 
Total number ratings: 18418 

App: New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition 
Total number ratings: 17588 

App: Merriam-Webster Dictionary 
Total number ratings: 16849 

App: Night Sky 
Total number ratings: 12122 

App: City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) 
Total number ratings: 8535 

App: LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools 
Total number ratings: 4693 

App: GUNS MODS for Minecraft PC Edition - Mods Tools 
Total number ratings: 1497 

App: Guides for Pokémon GO - Pokemon GO News and Cheats 
Total number ratings: 826 

App: WWDC 
Total number ratings: 762 

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating.

However, this niche seems to show some potential for the business. For example, taking another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, one could also embed a dictionary within the app, so users don't need to exit the app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea described above, but the other genres don't seem too interesting or useful for the business:

- Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

- Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope.

- Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge.

Now moving on to analyze the Google Play market a bit.

## Most Popular Apps by Genre on Google Play

For the Google Play market, the number of installs can be used to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [64]:
display_table(Google_final, 5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


One problem with this data is that it is not precise. For instance, if an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, I don't need very precise data for the purposes — I only want to get an idea which app genres attract the most users, and I don't need perfect precision with respect to the number of users.

I'm going to leave the numbers as they are, which means that I'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, I'll need to convert each install number to float — this means that I need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. I'll do this directly in the loop below, where I also compute the average number of installs for each genre (category).

In [92]:
Google_header

['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver']

In [93]:
# Frequency table for Category but I only use the keys
genres_Google = freq_table(Google_final, 1)

for key in genres_Google:
    total = 0
    len_genre = 0
    for item in Google_final:
        if item[1] == key:
            install = item[5].replace('+','')
            install = float(install.replace(',',''))
            total += install
            len_genre += 1
    genres_Google[key] = total / len_genre

In [96]:
genres_list = []
for key in genres_Google:
    genres_tuple = (genres_Google[key], key)
    genres_list.append(genres_tuple)

genres_list = sorted(genres_list, reverse=True)

for each in genres_list:
    print(each[1],':',each[0])

COMMUNICATION : 38456119.167247385
VIDEO_PLAYERS : 24727872.452830188
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17840110.40229885
PRODUCTIVITY : 16787331.344927534
GAME : 15588015.603248259
TRAVEL_AND_LOCAL : 13984077.710144928
ENTERTAINMENT : 11640705.88235294
TOOLS : 10801391.298666667
NEWS_AND_MAGAZINES : 9549178.467741935
BOOKS_AND_REFERENCE : 8767811.894736841
SHOPPING : 7036877.311557789
PERSONALIZATION : 5201482.6122448975
WEATHER : 5074486.197183099
HEALTH_AND_FITNESS : 4188821.9853479853
MAPS_AND_NAVIGATION : 4056941.7741935486
FAMILY : 3695641.8198090694
SPORTS : 3638640.1428571427
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
EDUCATION : 1833495.145631068
BUSINESS : 1712290.1474201474
LIFESTYLE : 1437816.2687861272
FINANCE : 1387692.475609756
HOUSE_AND_HOME : 1331540.5616438356
DATING : 854028.8303030303
COMICS : 817657.2727272727
AUTO_AND_VEHICLES : 647317.8170731707
LIBRARIES_AND_DEMO : 638503.734939759
PARENTING : 542603.6206896552
BEAUTY : 51315

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [97]:
for each in Google_final:
    if each[1] == "COMMUNICATION" and (each[5] == '1,000,000,000+'
                                      or each[5] == '500,000,000+'
                                      or each[5] == '100,000,000+'):
        print(each[0],':',each[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

I will remove all the communication apps that have over 100 million installs:

In [98]:
less_than_100m = []

for item in Google_final:
    install = item[5].replace('+','')
    install = float(install.replace(',',''))
    if (item[1] == 'COMMUNICATION') and (install < 100000000):
        less_than_100m.append(install)
        
print('Average installs after removing apps have more' 
      'than 100 million installs:', 
      sum(less_than_100m) / len(less_than_100m))

Average installs after removing apps have morethan 100 million installs: 3603485.3884615386


Same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where there are giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but this part of the market seems a bit saturated, so I'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since as described earlier this genre has some potential to work well on the App Store, and the goal is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [99]:
for each in Google_final:
    if each[1] == "BOOKS_AND_REFERENCE":
        print(each[0],':',each[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [100]:
for each in Google_final:
    if each[1] == "BOOKS_AND_REFERENCE" and (each[5] == '1,000,000,000+'
                                      or each[5] == '500,000,000+'
                                      or each[5] == '100,000,000+'):
        print(each[0],':',each[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [101]:
for each in Google_final:
    if each[1] == "BOOKS_AND_REFERENCE" and (each[5] == '1,000,000+'
                                      or each[5] == '5,000,000+'
                                      or each[5] == '10,000,000+'
                                      or each[5] == '50,000,000+'):
        print(each[0],':',each[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

Notice that there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so the business will need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

## Conclusions

In this project, I analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app that can be profitable for both markets.

The conclusion is that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so one needs to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.